# Imports 

In [9]:
import cv2
import os
import numpy as np
import json
from kafka import KafkaProducer, KafkaConsumer
from cassandra.cluster import Cluster
from PIL import Image
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import preprocess_input
import schedule
import time
import base64
import io
import requests
from keras.applications.vgg16 import VGG16  # Assuming you want to use Keras VGG16
from cassandra.util import uuid_from_time

# configuration

In [10]:
# Kafka configuration
bootstrap_servers = 'localhost:9092'
input_topic = 'photo-topic'
#output_topic = 'output_topicc'

# Cassandra configuration
cassandra_host = 'localhost'
cassandra_keyspace = 'big_data'#................................................................
cassandra_table = 'photo'

# Connect to Kafka
consumer = KafkaConsumer(bootstrap_servers=bootstrap_servers)
consumer.subscribe([input_topic])

# Kafka producer
#producer = KafkaProducer(bootstrap_servers=bootstrap_servers)


# Load the saved model
save_model = tf.keras.models.load_model('my_keras_model.h5')#.....................
class_names = ['fire_images', 'non_fire_images']
#image_folder = r"C:\Users\AL-FAJR\Desktop\Last_GP\FIRE-SMOKE-DATASET\Test\Neutral"

# Cassandra

In [11]:
# Connect to Cassandra
cluster = Cluster([cassandra_host])
session = cluster.connect()
#session.execute(f"CREATE KEYSPACE IF NOT EXISTS {cassandra_keyspace} WITH REPLICATION = {{ 'class' : 'SimpleStrategy', 'replication_factor' : 1 }}")
session.set_keyspace(cassandra_keyspace)
session.execute(f"CREATE TABLE IF NOT EXISTS {cassandra_table} (id UUID  ,homeId TEXT, roomId TEXT, class_name TEXT , image_Path TEXT , DateTime TIMESTAMP, PRIMARY KEY (roomId,DateTime,id) )")

# Kafka

In [12]:
# Consume messages from Kafka
for message in consumer:
    try:
        # Check if the message is empty
        if not message.value:
            print("Ignoring empty message.")
            continue

        # Print the content of the message
        print("Received message:", message.value)

        # Parse message as JSON
        data_dict = json.loads(message.value.decode('utf-8'))

        # Extract fields from the JSON
        HomeId = data_dict['HomeId']
        RoomId = data_dict['RoomId']
        DateTime = data_dict['DateTime']
        ImagePath = data_dict['ImagePath']
        print(ImagePath)
        # Read image from folder
        image_path = os.path.join(data_dict['ImagePath'])
        image = Image.open(image_path)

        # Preprocess image for VGG16
        image = image.resize((224, 224))
        image = image.convert('RGB')
        image_array = preprocess_input(np.array(image))
        image_array = np.expand_dims(image_array, axis=0)

        # Classify the image
        prediction = save_model.predict(image_array)
        class_index = np.where(prediction >= 0.5, 1, 0)[0][0]
        class_name = class_names[class_index]
        probability = float(prediction)
       

        session.execute(f"INSERT INTO {cassandra_table} (id,homeId ,roomId, class_name, image_Path, DateTime) VALUES (uuid(),'{HomeId}' ,'{RoomId}', '{class_name}', '{image_path}', '{DateTime}')")


        # Publish data to another Kafka topic
        data_dict['probability'] = probability
        data_dict['class_name'] = class_name

        # Convert the dictionary to JSON
       # json_data = json.dumps(data_dict)

        # Send the JSON data to the output topic
       # producer.send(output_topic, value=json_data.encode('utf-8'))

        print("Processed message:", data_dict)

    except Exception as e:
        print("Error processing message:", e)

# Close Kafka producer and consumer
producer.close()
consumer.close()


Received message: b'{"HomeId" : "268" , "RoomId" : "2471" , "DateTime": "2024-02-12 03:30:14" , "ImagePath" : "C:\\\\Users\\\\Moataz Nasr\\\\Downloads\\\\images\\\\image_26.jpg"}'
C:\Users\Moataz Nasr\Downloads\images\image_26.jpg
1/1 [==============================] - 1s 662ms/step
Processed message: {'HomeId': '268', 'RoomId': '2471', 'DateTime': '2024-02-12 03:30:14', 'ImagePath': 'C:\\Users\\Moataz Nasr\\Downloads\\images\\image_26.jpg', 'probability': 4.374553423821279e-28, 'class_name': 'fire_images'}


C:\Users\Moataz Nasr\AppData\Local\Temp\ipykernel_29284\4174358698.py:35: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  probability = float(prediction)


Received message: b'{"HomeId" : "268" , "RoomId" : "2471" , "DateTime": "2024-02-12 03:31:02" , "ImagePath" : "C:\\\\Users\\\\Moataz Nasr\\\\Downloads\\\\images\\\\image_439.jpg"}'
C:\Users\Moataz Nasr\Downloads\images\image_439.jpg
1/1 [==============================] - 0s 233ms/step
Processed message: {'HomeId': '268', 'RoomId': '2471', 'DateTime': '2024-02-12 03:31:02', 'ImagePath': 'C:\\Users\\Moataz Nasr\\Downloads\\images\\image_439.jpg', 'probability': 4.420784710097876e-29, 'class_name': 'fire_images'}
Received message: b'{"HomeId" : "268" , "RoomId" : "2471" , "DateTime": "2024-02-12 03:31:26" , "ImagePath" : "C:\\\\Users\\\\Moataz Nasr\\\\Downloads\\\\images\\\\image_530.jpg"}'
C:\Users\Moataz Nasr\Downloads\images\image_530.jpg
1/1 [==============================] - 0s 263ms/step
Processed message: {'HomeId': '268', 'RoomId': '2471', 'DateTime': '2024-02-12 03:31:26', 'ImagePath': 'C:\\Users\\Moataz Nasr\\Downloads\\images\\image_530.jpg', 'probability': 0.0, 'class_name': '

C:\Users\Moataz Nasr\AppData\Local\Programs\Python\Python311\Lib\site-packages\PIL\TiffImagePlugin.py:866: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


1/1 [==============================] - 0s 272ms/step
Processed message: {'HomeId': '268', 'RoomId': '2471', 'DateTime': '2024-02-12 03:35:21', 'ImagePath': 'C:\\Users\\Moataz Nasr\\Downloads\\images\\image_9.jpg', 'probability': 1.0, 'class_name': 'non_fire_images'}
Received message: b'{"HomeId" : "268" , "RoomId" : "2471" , "DateTime": "2024-02-12 03:35:21" , "ImagePath" : "C:\\\\Users\\\\Moataz Nasr\\\\Downloads\\\\images\\\\image_7.jpg"}'
C:\Users\Moataz Nasr\Downloads\images\image_7.jpg
1/1 [==============================] - 0s 205ms/step
Processed message: {'HomeId': '268', 'RoomId': '2471', 'DateTime': '2024-02-12 03:35:21', 'ImagePath': 'C:\\Users\\Moataz Nasr\\Downloads\\images\\image_7.jpg', 'probability': 1.0, 'class_name': 'non_fire_images'}


KeyboardInterrupt: 